In [37]:
# Load scikit's random forest classifier library
from sklearn.ensemble import RandomForestClassifier

# Load pandas
import pandas as pd

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import os
import json

import numpy as np

In [19]:
all_files = os.listdir('JSONs\JSONs')
len(all_files)

137886

In [20]:
files_2d = os.listdir('2d/2d')
len(files_2d)

2961

In [21]:
files_1d = os.listdir('1d/1d')
len(files_1d)

772

In [22]:
def process_dict(data, properties = {}):
    for k in data.keys():
        if isinstance(data.get(k),float) or isinstance(data.get(k),int) or isinstance(data.get(k),bool):
            properties[k]= data.get(k)
        
        if isinstance(data.get(k), dict):
            temp = data.get(k)
            properties = process_dict(temp,properties)
            
        if isinstance(data.get(k), list):
            if len(data.get(k)) and isinstance(data.get(k)[0], dict):
                properties = process_dict(data.get(k)[0],properties)
                
    return properties

In [23]:
all_files[4]

'mp-1000.json'

In [24]:
materials = []
'''
for file in all_files:
    with open('JSONs\JSONs\{}'.format(file)) as f:
        data = json.loads(f.read())
    properties = process_dict(data, {})
    properties['name'] = file.split('.')[0]
    properties['class'] = '2D'
    materials.append(properties)
'''

for file in files_2d:
    with open('2d/2d/{}'.format(file)) as f:
        data = json.loads(f.read())
    properties = process_dict(data, {})
    properties['name'] = file.split('.')[0]
    properties['class'] = '2D'
    materials.append(properties)

for file in files_1d:
    with open('1d/1d/{}'.format(file)) as f:
        data = json.loads(f.read())
    properties = process_dict(data, {})
    properties['name'] = file.split('.')[0]
    properties['class'] = '1D'
    materials.append(properties)

'''
with open('JSONs\JSONs\{}'.format(all_files[4])) as f:
        data = json.loads(f.read())
properties = process_dict(data, {})
properties['name'] = all_files[4].split('.')[0]
properties['class'] = '2D'
materials.append(properties)
'''

"\nwith open('JSONs\\JSONs\\{}'.format(all_files[4])) as f:\n        data = json.loads(f.read())\nproperties = process_dict(data, {})\nproperties['name'] = all_files[4].split('.')[0]\nproperties['class'] = '2D'\nmaterials.append(properties)\n"

In [25]:
print(len(materials))

3733


In [27]:
df = pd.DataFrame(materials)
df.set_index('name',inplace=True)
df.head(3)

,A,B,band_gap,density,deprecated,e_above_hull,efermi,encut,final_energy,final_energy_per_atom,...,LMAXTAU,weight,NGY,NGX,Ac,SYMPREC,Pa,structureid,NGZ,Rb
name,,,,,,,,,,,,,,,,,,,,,
mp-10009,1.0,1.0,0.898,4.804723,False,0.00000,2.675483,520.0,-27.598327,-3.449791,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp-10014,1.0,2.0,0.000,6.161496,False,0.00033,3.394650,520.0,-23.275075,-7.758358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mp-10021,1.0,NaN,0.000,6.092337,False,0.01726,3.458981,520.0,-6.021659,-3.010830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
df.columns

Index(['A', 'B', 'band_gap', 'density', 'deprecated', 'e_above_hull', 'efermi',
       'encut', 'final_energy', 'final_energy_per_atom',
       ...
       'LMAXTAU', 'weight', 'NGY', 'NGX', 'Ac', 'SYMPREC', 'Pa', 'structureid',
       'NGZ', 'Rb'],
      dtype='object', length=184)

In [28]:
bool_columns = list(df.dtypes[df.dtypes==bool].index)
for col in bool_columns:
    df[col] = df[col].astype(int)

In [29]:
# 1 means 2D, 0 means 1D
df['class'] = df['class'].astype('category').cat.codes

In [40]:
scaler = StandardScaler()
X = df[list(set(df.columns)-set(['class']))]
y= df['class']
X = X.replace(np.nan,0)
X_norm = scaler.fit_transform(X)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60)

In [73]:

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [74]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=2, random_state=0)

In [75]:
# View a list of the features and their importance scores (how important was this feature to the decision)
y_pred = clf.predict(X_test)
print(*list(zip(df.columns, clf.feature_importances_ * 100)), sep = "\n")
#print(confusion_matrix(y_test, y_pred))
# Create confusion matrix
print(pd.crosstab(y_test, y_pred, rownames=['Actual'], colnames=['Predicted']))
print(classification_report(y_test, y_pred))

('A', 0.11492562601954653)
('B', 0.0)
('band_gap', 0.0)
('density', 1.5272742727614208)
('deprecated', 2.074590102101393)
('e_above_hull', 0.057676611979812296)
('efermi', 0.08898826954968439)
('encut', 0.18911963571339271)
('final_energy', 0.0)
('final_energy_per_atom', 0.26517797829161716)
('formation_energy_per_atom', 2.6862957110251378)
('has_bandstructure', 0.5731976781560556)
('a', 3.3163731086934525)
('b', 0.33534002308574584)
('c', 0.0)
('alpha', 0.13673650615202204)
('beta', 0.3354129182874799)
('gamma', 0.39795913668612004)
('volume', 3.304281491906725)
('occu', 0.5576192077184549)
('ISIF', 0.0)
('IBRION', 0.15671230785038967)
('NSW', 0.0)
('NELM', 2.1538268763557524)
('ISPIN', 0.13005428795722715)
('ENCUT', 0.021279725317812304)
('EDIFF', 0.48528291937287016)
('NELMIN', 2.9052093164180115)
('ISMEAR', 0.16355218798321805)
('SIGMA', 0.14842062106055073)
('LWAVE', 0.21038447063047389)
('ICHARG', 0.14365811826513317)
('NPAR', 1.2592996015591442)
('nkpoints', 0.10348009306933609)